In [1]:
from dotenv import load_dotenv
load_dotenv()

True

initialising and invoking the model

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model = "gemini-2.5-flash-lite")

response = model.invoke("capital of telangana")
response

AIMessage(content='The capital of Telangana is **Hyderabad**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='lc_run--019bbd9e-aa93-7a81-994b-5c23e91240b0-0', tool_calls=[], invalid_tool_calls=[])

In [3]:
from langchain.agents import create_agent

agent = create_agent(model = model)


In [7]:
from langchain.messages import HumanMessage #represnts input from a human user in a conversation

message = HumanMessage(content= "What is the capital of India")

response = agent.invoke(
    {
        "messages" : message
    }
)

print(response)

{'messages': [HumanMessage(content='What is the capital of India', additional_kwargs={}, response_metadata={}, id='74dba551-6e1b-4347-8e20-ca9c24d1a8b0'), AIMessage(content='The capital of India is **New Delhi**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='lc_run--019bbda1-bf63-7982-98f3-b2ba5fdfb847-0', tool_calls=[], invalid_tool_calls=[])]}


In [ ]:
from pprint import pprint # normal print just gives you raw text but pprint will give ouput in a indented way for better readability
pprint(response)

{'messages': [HumanMessage(content='What is the capital of India', additional_kwargs={}, response_metadata={}, id='74dba551-6e1b-4347-8e20-ca9c24d1a8b0'),
              AIMessage(content='The capital of India is **New Delhi**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='lc_run--019bbda1-bf63-7982-98f3-b2ba5fdfb847-0', tool_calls=[], invalid_tool_calls=[])]}


In [11]:
response['messages'][-1].content

'The capital of India is **New Delhi**.'

In [ ]:
from langchain.messages import AIMessage

response = agent.invoke(
    {'messages' :
        [
            HumanMessage(content='What is the capital of India'),
            AIMessage(content='The capital of India is **New Delhi**'),
            HumanMessage(content='Tell me a fact about capital city')
        ]
        }
)

pprint(response)

{'messages': [HumanMessage(content='What is the capital of India', additional_kwargs={}, response_metadata={}, id='0dad8d90-cdf8-487c-b1d4-498a6da877c8'),
              AIMessage(content='The capital of India is **New Delhi**', additional_kwargs={}, response_metadata={}, id='de9dafff-149c-4b7c-a8da-288db39703bb', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content='Tell me a fact about capital city', additional_kwargs={}, response_metadata={}, id='397731c1-11a5-442b-84b4-c9b1e54045b6'),
              AIMessage(content="Here's a fact about New Delhi:\n\nNew Delhi is not just a city; it's also a **district within the larger metropolitan area of Delhi**. This means that while New Delhi is the political capital and houses the government, the entire vast urban agglomeration known as Delhi is a Union Territory with its own administration.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'mo

Streaming output

In [ ]:
#instead of waiting for compelete response , agent.stream enables streaming the response in real-time incremnetally
for token, metadata in agent.stream(
    { 'messages' :
        [
            HumanMessage(content='What is the capital of India'),
            AIMessage(content='The capital of India is **New Delhi**'),
            HumanMessage(content='Tell me a fact about capital city')
        ]
    }, stream_mode="messages"
):
    
    if token.content:
        print(token.content,end = "",flush = True)

Here's a fact about New Delhi:

**New Delhi is not an independent city but rather a district within the larger metropolitan area of Delhi.** While it's the seat of the Indian government and its most recognizable capital, the entire National Capital Territory of Delhi is a vast urban expanse.

prompting with structured output

In [19]:
from pydantic import BaseModel

class StrucutredOutput(BaseModel):
    name : str
    location : str
    vibe : str
    economy: int
    colour :str
    story :str

agent2 = create_agent(
    model = model,
    system_prompt= " imagine you are christopher nolan, , create a city at the users rewust",
    response_format=StrucutredOutput
)

question = HumanMessage(content = "what is the capital")

response = agent2.invoke({
    'messages' : [question]
}
)
response

{'messages': [HumanMessage(content='what is the capital', additional_kwargs={}, response_metadata={}, id='bb3e31d4-d921-440a-9594-22ca052b37f2'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'StrucutredOutput', 'arguments': '{"location": "The Shard", "colour": "Black", "economy": 1000000.0, "story": "A city built on the precipice of a great discovery, shrouded in mystery and ambition.", "vibe": "Ominous", "name": "The Citadel"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='lc_run--019bbdc2-eab8-7601-a84c-d8985caef1e6-0', tool_calls=[{'name': 'StrucutredOutput', 'args': {'location': 'The Shard', 'colour': 'Black', 'economy': 1000000.0, 'story': 'A city built on the precipice of a great discovery, shrouded in mystery and ambition.', 'vibe': 'Ominous', 'name': 'The Citadel'}, 'id': 'c727f0c7-be4b-4a1e-b9af-d040dd63b776', 'type': 'tool_ca

In [20]:
response['structured_response']

StrucutredOutput(name='The Citadel', location='The Shard', vibe='Ominous', economy=1000000, colour='Black', story='A city built on the precipice of a great discovery, shrouded in mystery and ambition.')

In [22]:
capital_info = response['structured_response']

print("Name:", capital_info.name)
print("Location:", capital_info.location)
print("Vibe:", capital_info.vibe)
print("Economy:", capital_info.economy)
print("Colour:", capital_info.colour)
print("Story:", capital_info.story)

Name: The Citadel
Location: The Shard
Vibe: Ominous
Economy: 1000000
Colour: Black
Story: A city built on the precipice of a great discovery, shrouded in mystery and ambition.
